# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [ ]:
import json

In [ ]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [ ]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [ ]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [ ]:
print('notice that the instructor and code are tuples now')
courses[1246]

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [ ]:
#Answers for PA1-Q5

a) how many faculty taught COSI courses last year?

In [ ]:
len({tuple(course['instructor']) for course in courses if course['subject'] == 'COSI'})

b)what is the total number of students taking COSI courses last year?

In [ ]:
student_cs = [course['enrolled'] for course in courses if course['subject'] == 'COSI']
sum([i for i in student_cs])

c) what was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [ ]:
import statistics as st
st.median([int(course['enrolled']) for course in courses if course['subject'] == 'COSI' and course['enrolled']>10])

d) create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [ ]:
from itertools import groupby
subject_and_enrolled = [(k, sum([i["enrolled"] for i in list(v)])) for k,v in groupby(courses,key=lambda courses:courses['subject'])]
subject_and_enrolled.sort(key = lambda subject_and_enrolled: -subject_and_enrolled[1])
print(subject_and_enrolled[:10])

e)do the same as in (d) but print the top 10 subjects in terms of number of courses offered

In [ ]:
from collections import Counter
course_by_subject = sorted(courses, key = lambda course: course['subject'])
subject = [c['subject'] for c in course_by_subject]
Counter(subject).most_common()[:10]

f)do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject

In [ ]:
instructor = list({(c['subject'],c['instructor']) for c in courses})
instructor.sort()
lst = list()
for key, val in instructor :
    lst.append(key)
Counter(lst).most_common()[:10]

g)list the top 20 faculty in terms of number of students they taught

In [ ]:
course_by_size = sorted(courses, key = lambda course: -course['enrolled'])
[(c['enrolled'], c['instructor']) for c in course_by_size[:20]] 


h)list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)

In [ ]:
coures_by_student = sorted(courses, key = lambda course: -course['enrolled'])
[(c['enrolled'], c['subject'], c['coursenum']) for c in course_by_size[:20]] 

i)

Ni Jian): list the top 3 instructors in terms of the number of students they taught in subject COSI

In [ ]:
course_by_size = sorted((c for c in courses if c['subject'] =='COSI'), key = lambda course: -course['enrolled'])
[(c['enrolled'], c['instructor']) for c in course_by_size[:3]]                    

Lu Lu): List all the classes that meet on Mondays and Wednesdays.

In [ ]:
# len([course for course in courses if 'mon' in course['times'][0]['days']])
for course in courses:
    if course['times']:
        if 'm'  in course['times'][0]['days'] and 'w' in course['times'][0]['days']:
            print(course['name'])

Wantong Lin):

Vu Le): List all the classes that have open sections:

In [ ]:
listing = ((course['subject'],course['coursenum']) for course in courses if course['status_text'] == 'Open')
for item in listing:
    print(item)

Mahmoud Salah):

Ni Jian):